In [1]:
import os
import sys
import math
import sklearn
import matplotlib


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
import statsmodels.api as sm


from tensorflow import keras
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso
from sklearn.utils.validation import column_or_1d
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler

In [2]:
CSV_PATH = "complete-frame.csv"
CSV_MINER_PATH = "testminereffectiveness.csv"

In [213]:
def label_rename1 (row):
    return row['path_test'].split('/')[len(row['path_test'].split('/')) - 1].split('.')[0]

def label_rename2 (row):
    return row['path_src'].split('/')[len(row['path_src'].split('/')) - 1].split('.')[0]

def load_quartile(frame):
    low, high = frame.mutation.quantile([0.25,0.75])
    frame_low = frame.query('mutation<{low}'.format(low=low))
    frame_high = frame.query('mutation>{high}'.format(high=high))
    frame_low['mutation'] = 0
    frame_high['mutation'] = 1
    frame = pd.concat([frame_low, frame_high], ignore_index=True)
    frame = frame.sample(frac=1).reset_index(drop=True)
    return frame;

def load_frame():
    
    d = {'TestClassName' : 'ClassName', 'Vocabulary' : 'Vocabulary_prod', 'Non Whithe Characters' : 'Non Whithe Characters_prod',
     'No. Method Invoctions' : 'No. Method Invoctions_prod',
    'AST size' : 'AST size_prod', 'Max Depth' : 'Max Depth_prod', 'Sum Depths' : 'Sum Depths_prod',
     'Avg Depth' : 'Avg Depth_prod', 'Branching' : 'Branching_prod', 'Dexterity' : 'Dexterity_prod',
    'No. Expressions' : 'No. Expressions_prod', 'No. Try' : 'No. Try_prod', 'No. Catch' : 'No. Catch_prod',
     'No. Loop' : 'No. Loop_prod', 'No. Conditions' : 'No. Conditions_prod', 'No. Else' : 'No. Else_prod'}
    
    frame1 = pd.read_csv(CSV_PATH, sep=",")
    frame1 = frame1.sample(frac=1).reset_index(drop=True)
    frame1['TestClassName'] = frame1.apply(lambda row: label_rename1(row), axis=1)
    frame1['ClassName'] = frame1.apply(lambda row: label_rename2(row), axis=1)
        
    
    frame2 = pd.read_csv(CSV_MINER_PATH, sep=',')
    
    frame3 = pd.read_csv(CSV_MINER_PATH, sep=',')
    frame3 = frame3.rename(columns = d, errors = 'raise')
    frame3 = frame3.drop(['No. Tests', 'Bad API', 'Junit', 'Hamcrest', 'Mockito', 'Nº','Project'], axis=1)
    
    
    frame = pd.merge(frame1, frame2, on='TestClassName')
    
    frame = pd.merge(frame, frame3, on='ClassName')
    

    frame = frame.drop(['project', 'module', 'path_test','test_name','path_src',
                        'commit', 'class_name'], axis=1)
    frame = frame.sample(frac=1).reset_index(drop=True)
    frame = frame.dropna()

    return frame


def load_all_test_data_dynamic(frame):
    columns = ['isAssertionRoulette',
       'isEagerTest', 'isLazyTest', 'isMysteryGuest',
       'isSensitiveEquality', 'isResourceOptimism', 'isForTestersOnly',
       'isIndirectTesting', 'LOC_test', 'HALSTEAD_test', 'RFC_test', 'CBO_test', 'MPC_test', 'IFC_test',
       'DAC_test', 'DAC2_test', 'LCOM1_test', 'LCOM2_test', 'LCOM3_test',
       'LCOM4_test', 'CONNECTIVITY_test', 'LCOM5_test', 'COH_test',
       'TCC_test', 'LCC_test', 'ICH_test', 'WMC_test', 'NOA_test',
       'NOPA_test', 'NOP_test', 'McCABE_test', 'BUSWEIMER_test', 'test_readability', 'No. Tests',
       'Vocabulary', 'Non Whithe Characters', 'No. Method Invoctions',
       'AST size', 'Max Depth', 'Sum Depths', 'Avg Depth', 'Branching',
       'Dexterity', 'No. Expressions', 'No. Try', 'No. Catch', 'No. Loop',
       'No. Conditions', 'No. Else', 'Bad API', 'Junit', 'Hamcrest',
       'Mockito'] # 'line_coverage']

    data_x = frame[columns].round(2)
    data_y = pd.concat([frame.mutation], axis = 1)
    return data_x, data_y, len(columns)


def load_all_prod_data(frame):
    columns = ['LOC_prod', 'HALSTEAD_prod', 'RFC_prod',
       'CBO_prod', 'MPC_prod', 'IFC_prod', 'DAC_prod', 'DAC2_prod',
       'LCOM1_prod', 'LCOM2_prod', 'LCOM3_prod', 'LCOM4_prod',
       'CONNECTIVITY_prod', 'LCOM5_prod', 'COH_prod', 'TCC_prod',
       'LCC_prod', 'ICH_prod', 'WMC_prod', 'NOA_prod', 'NOPA_prod',
       'NOP_prod', 'McCABE_prod', 'BUSWEIMER_prod',    
       'csm_CDSBP', 'csm_CC', 'csm_FD', 'csm_Blob', 'csm_SC', 'csm_MC',
       'csm_LM', 'csm_FE', 'prod_readability', 'Vocabulary_prod', 'Non Whithe Characters_prod',
       'No. Method Invoctions_prod', 'AST size_prod', 'Max Depth_prod',
       'Sum Depths_prod', 'Avg Depth_prod', 'Branching_prod',
       'Dexterity_prod', 'No. Expressions_prod', 'No. Try_prod',
       'No. Catch_prod', 'No. Loop_prod', 'No. Conditions_prod',
       'No. Else_prod']

    data_x = frame[columns].round(2)
    data_y = pd.concat([frame.mutation], axis = 1)
    return data_x, data_y, len(columns)


def load_all_data(frame):
    columns = ['isAssertionRoulette',
       'isEagerTest', 'isLazyTest', 'isMysteryGuest',
       'isSensitiveEquality', 'isResourceOptimism', 'isForTestersOnly',
       'isIndirectTesting', 'LOC_prod', 'HALSTEAD_prod', 'RFC_prod',
       'CBO_prod', 'MPC_prod', 'IFC_prod', 'DAC_prod', 'DAC2_prod',
       'LCOM1_prod', 'LCOM2_prod', 'LCOM3_prod', 'LCOM4_prod',
       'CONNECTIVITY_prod', 'LCOM5_prod', 'COH_prod', 'TCC_prod',
       'LCC_prod', 'ICH_prod', 'WMC_prod', 'NOA_prod', 'NOPA_prod',
       'NOP_prod', 'McCABE_prod', 'BUSWEIMER_prod', 'LOC_test',
       'HALSTEAD_test', 'RFC_test', 'CBO_test', 'MPC_test', 'IFC_test',
       'DAC_test', 'DAC2_test', 'LCOM1_test', 'LCOM2_test', 'LCOM3_test',
       'LCOM4_test', 'CONNECTIVITY_test', 'LCOM5_test', 'COH_test',
       'TCC_test', 'LCC_test', 'ICH_test', 'WMC_test', 'NOA_test',
       'NOPA_test', 'NOP_test', 'McCABE_test', 'BUSWEIMER_test',
       'csm_CDSBP', 'csm_CC', 'csm_FD', 'csm_Blob', 'csm_SC', 'csm_MC',
       'csm_LM', 'csm_FE', 'prod_readability', 'test_readability', 'No. Tests',
       'Vocabulary', 'Non Whithe Characters', 'No. Method Invoctions',
       'AST size', 'Max Depth', 'Sum Depths', 'Avg Depth', 'Branching',
       'Dexterity', 'No. Expressions', 'No. Try', 'No. Catch', 'No. Loop',
       'No. Conditions', 'No. Else', 'Bad API', 'Junit', 'Hamcrest',
       'Mockito', 'Vocabulary_prod', 'Non Whithe Characters_prod',
       'No. Method Invoctions_prod', 'AST size_prod', 'Max Depth_prod',
       'Sum Depths_prod', 'Avg Depth_prod', 'Branching_prod',
       'Dexterity_prod', 'No. Expressions_prod', 'No. Try_prod',
       'No. Catch_prod', 'No. Loop_prod', 'No. Conditions_prod',
       'No. Else_prod']

    data_x = frame[columns].round(2)
    data_y = pd.concat([frame.mutation], axis = 1)
    return data_x, data_y, len(columns)

In [197]:
load_all_data(load_frame())

(      isAssertionRoulette  isEagerTest  isLazyTest  isMysteryGuest  \
 0                       1            1           0               0   
 1                       1            1           0               0   
 2                       0            0           0               1   
 3                       0            0           0               0   
 4                       1            0           0               0   
 ...                   ...          ...         ...             ...   
 2678                    1            0           0               0   
 2679                    1            1           0               0   
 2680                    1            1           0               0   
 2681                    1            0           0               0   
 2682                    1            0           0               0   
 
       isSensitiveEquality  isResourceOptimism  isForTestersOnly  \
 0                       0                   0                 0   
 1        

In [175]:
frame1 = pd.read_csv('good_tests.csv', sep=",")
frame1['TestClassName'] = frame1.apply(lambda row: label_rename1(row), axis=1)
frame2 = load_frame() #pd.read_csv(CSV_MINER_PATH, sep=',')

frame = frame2[frame2['TestClassName'].isin(frame1['TestClassName'].to_list())]
frame = frame.drop(['TestClassName', 'ClassName', 'Project'], axis=1)
frame.to_csv ('good_tests_extended.csv', index = False, header=True)
frame

,class_name,mutation,no_mutations,line_coverage,isAssertionRoulette,isEagerTest,isLazyTest,isMysteryGuest,isSensitiveEquality,isResourceOptimism,...,Sum Depths_prod,Avg Depth_prod,Branching_prod,Dexterity_prod,No. Expressions_prod,No. Try_prod,No. Catch_prod,No. Loop_prod,No. Conditions_prod,No. Else_prod
3,com.google.debugging.sourcemap.Base64,0.933333,30,0.941176,1,1,0,0,0,0,...,2670.0,2.792887,0.995816,37.0,792.0,0.0,0.0,16.0,9.0,0.0
14,com.google.javascript.jscomp.ImplicitNullabili...,0.950000,20,1.000000,0,0,0,0,0,0,...,917.0,3.322464,1.663043,35.0,216.0,0.0,0.0,2.0,8.0,0.0
16,com.google.javascript.jscomp.RecordFunctionInf...,0.956522,46,0.961538,1,1,0,0,0,0,...,449.0,3.680328,1.991803,28.0,133.0,0.0,0.0,0.0,2.0,0.0
17,io.reactivex.internal.operators.flowable.Flowa...,1.000000,8,1.000000,1,0,0,0,1,0,...,1323.0,4.955056,3.411985,33.0,322.0,1.0,1.0,0.0,3.0,1.0
18,com.google.javascript.jscomp.InlineObjectLiterals,1.000000,6,1.000000,0,0,0,0,0,0,...,3216.0,2.644737,1.156250,39.0,943.0,0.0,0.0,13.0,29.0,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2649,io.reactivex.internal.operators.maybe.MaybeIgn...,1.000000,2,1.000000,0,0,0,0,0,0,...,474.0,5.096774,3.204301,27.0,119.0,0.0,0.0,0.0,1.0,0.0
2650,org.apache.commons.collections4.set.MapBackedSet,0.969072,97,0.968750,0,0,0,0,0,0,...,956.0,5.031579,4.789474,35.0,252.0,0.0,0.0,1.0,1.0,0.0
2658,com.google.javascript.jscomp.ClosureOptimizePr...,0.927419,248,1.000000,0,0,0,0,0,0,...,1819.0,2.824534,1.397516,33.0,532.0,0.0,0.0,4.0,20.0,6.0
2663,io.reactivex.internal.operators.observable.Obs...,1.000000,3,1.000000,1,0,0,0,0,0,...,726.0,4.624204,2.490446,28.0,168.0,1.0,1.0,0.0,5.0,1.0


In [176]:
frame1 = pd.read_csv('bad_tests.csv', sep=",")
frame1['TestClassName'] = frame1.apply(lambda row: label_rename1(row), axis=1)
frame2 = load_frame() #pd.read_csv(CSV_MINER_PATH, sep=',')

frame = frame2[frame2['TestClassName'].isin(frame1['TestClassName'].to_list())]
frame = frame.drop(['TestClassName', 'ClassName', 'Project'], axis=1)


frame.to_csv ('bad_tests_extended.csv', index = False, header=True)
frame

,class_name,mutation,no_mutations,line_coverage,isAssertionRoulette,isEagerTest,isLazyTest,isMysteryGuest,isSensitiveEquality,isResourceOptimism,...,Sum Depths_prod,Avg Depth_prod,Branching_prod,Dexterity_prod,No. Expressions_prod,No. Try_prod,No. Catch_prod,No. Loop_prod,No. Conditions_prod,No. Else_prod
0,org.apache.commons.lang3.reflect.TypeLiteral,0.363636,44,0.750000,1,1,0,0,0,0,...,2449.0,3.139744,1.524359,48.0,623.0,0.0,0.0,3.0,16.0,9.0
2,org.opengrok.suggest.query.SuggesterWildcardQuery,0.357143,14,0.500000,0,0,0,0,0,0,...,145.0,5.178571,3.428571,21.0,36.0,0.0,0.0,0.0,1.0,0.0
4,org.apache.commons.math3.exception.TooManyEval...,0.333333,3,1.000000,0,0,0,0,0,0,...,64.0,5.818182,4.272727,18.0,16.0,0.0,0.0,0.0,0.0,0.0
8,com.puppycrawl.tools.checkstyle.checks.imports...,0.454545,22,1.000000,0,0,0,0,0,0,...,171.0,4.275000,2.650000,23.0,48.0,0.0,0.0,0.0,1.0,1.0
9,org.apache.commons.math3.random.RandomAdaptor,0.365854,41,0.695652,1,1,0,0,0,0,...,464.0,4.989247,5.387097,29.0,116.0,0.0,0.0,0.0,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2667,org.jfree.data.xy.MatrixSeriesCollection,0.179487,117,0.316667,1,1,0,0,0,0,...,1266.0,3.180905,2.120603,42.0,406.0,0.0,0.0,1.0,7.0,0.0
2671,org.jsoup.parser.Tokeniser,0.381963,377,0.579618,1,0,0,0,0,0,...,2579.0,3.332041,3.325581,42.0,753.0,1.0,1.0,2.0,28.0,9.0
2675,com.google.debugging.sourcemap.Util,0.134615,156,0.215686,1,1,0,0,1,0,...,1200.0,3.428571,2.114286,37.0,314.0,0.0,0.0,3.0,8.0,0.0
2680,io.reactivex.Observable,0.084152,4171,0.152627,1,1,0,0,0,0,...,52986.0,5.226990,15.525106,49.0,18616.0,3.0,4.0,1.0,54.0,6.0


In [185]:
import tensorflow
from numpy.random import seed
from sklearn.model_selection import KFold


seed(1)
tensorflow.random.set_seed(2)


frame = load_frame()
data_x, data_y, number_of_features = load_all_data(frame) #load_meaningful_subset_static(frame)
data_y = pd.concat([frame.mutation], axis = 1).round(2) #.mul(10)


data_x = data_x.values
data_y = data_y.values

scaler = StandardScaler()
scaler.fit(data_x)
data_x = scaler.transform(data_x)
    
seed = 7
np.random.seed(seed)
kfold = KFold(n_splits=10, shuffle=False, random_state=seed)
cvscores = []
overscores = []

for train, test in kfold.split(data_x, data_y):

    #x_validation, x_test, y_validation, y_test = train_test_split(data_x[validation_and_test], data_y[validation_and_test], test_size=.5, random_state=seed)


    model = keras.Sequential()
    model.add(keras.layers.Dense(number_of_features, activation='relu', input_dim=number_of_features))
    model.add(keras.layers.Dense(40, activation='relu', kernel_regularizer= keras.regularizers.l2(0.01)))
    model.add(keras.layers.Dense(20, activation='relu', kernel_regularizer= keras.regularizers.l2(0.01)))
    model.add(keras.layers.Dense(1))

    model.compile(optimizer='adam',
                  loss='mean_squared_error',
                  metrics=['mae'])


    #early_stopping_monitor = keras.callbacks.EarlyStopping(patience=1,restore_best_weights=True)

    history = model.fit(data_x[train], data_y[train], epochs=500, verbose=1)#,
                        #validation_data=(x_validation, y_validation), callbacks=[early_stopping_monitor])


        
    scores = model.evaluate(data_x[test], data_y[test], verbose=0)
    overfit = model.evaluate(data_x[train], data_y[train], verbose=0)
    print('{}: {}'.format(model.metrics_names[1], scores[1]))
    print('Overfit {}: {}'.format(model.metrics_names[1], overfit[1]))
    #print("-------------------------------------------")

    cvscores.append(scores[1])
    overscores.append(overfit[1])
    
print("-------------------------------------------")
print("Final results")
print('{} (+/- {})'.format(np.mean(cvscores), np.std(cvscores)))
print('And overfit of {} (+/- {})'.format(np.mean(overscores), np.std(overscores)))
print("-------------------------------------------")

/Users/Dor/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


Train on 2412 samples
Epoch 1/500
2412/2412 [==============================] - 1s 393us/sample - loss: 0.9012 - mae: 0.2946
Epoch 2/500
2412/2412 [==============================] - 0s 91us/sample - loss: 0.6057 - mae: 0.1584
Epoch 3/500
2412/2412 [==============================] - 0s 95us/sample - loss: 0.4762 - mae: 0.1299
Epoch 4/500
2412/2412 [==============================] - 0s 91us/sample - loss: 0.3806 - mae: 0.1171
Epoch 5/500
2412/2412 [==============================] - 0s 85us/sample - loss: 0.3086 - mae: 0.1113
Epoch 6/500
2412/2412 [==============================] - 0s 85us/sample - loss: 0.2523 - mae: 0.1053
Epoch 7/500
2412/2412 [==============================] - 0s 87us/sample - loss: 0.2086 - mae: 0.1022
Epoch 8/500
2412/2412 [==============================] - 0s 85us/sample - loss: 0.1739 - mae: 0.0988
Epoch 9/500
2412/2412 [==============================] - 0s 89us/sample - loss: 0.1461 - mae: 0.0981
Epoch 10/500
2412/2412 [==============================] - 0s 92us/sa

2412/2412 [==============================] - 0s 91us/sample - loss: 0.0117 - mae: 0.0684
Epoch 82/500
2412/2412 [==============================] - 0s 94us/sample - loss: 0.0115 - mae: 0.0689
Epoch 83/500
2412/2412 [==============================] - 0s 106us/sample - loss: 0.0116 - mae: 0.0685
Epoch 84/500
2412/2412 [==============================] - 0s 102us/sample - loss: 0.0114 - mae: 0.0682
Epoch 85/500
2412/2412 [==============================] - 0s 96us/sample - loss: 0.0112 - mae: 0.0669
Epoch 86/500
2412/2412 [==============================] - 0s 93us/sample - loss: 0.0110 - mae: 0.0655
Epoch 87/500
2412/2412 [==============================] - 0s 93us/sample - loss: 0.0111 - mae: 0.0664
Epoch 88/500
2412/2412 [==============================] - 0s 93us/sample - loss: 0.0111 - mae: 0.0664
Epoch 89/500
2412/2412 [==============================] - 0s 94us/sample - loss: 0.0110 - mae: 0.0662
Epoch 90/500
2412/2412 [==============================] - 0s 101us/sample - loss: 0.0112 - ma

2412/2412 [==============================] - 0s 108us/sample - loss: 0.0055 - mae: 0.0433
Epoch 240/500
2412/2412 [==============================] - 0s 103us/sample - loss: 0.0056 - mae: 0.0449
Epoch 241/500
2412/2412 [==============================] - 0s 93us/sample - loss: 0.0054 - mae: 0.0422
Epoch 242/500
2412/2412 [==============================] - 0s 93us/sample - loss: 0.0055 - mae: 0.0436
Epoch 243/500
2412/2412 [==============================] - 0s 100us/sample - loss: 0.0059 - mae: 0.0472
Epoch 244/500
2412/2412 [==============================] - 0s 97us/sample - loss: 0.0058 - mae: 0.0450
Epoch 245/500
2412/2412 [==============================] - 0s 107us/sample - loss: 0.0055 - mae: 0.0440
Epoch 246/500
2412/2412 [==============================] - 0s 104us/sample - loss: 0.0060 - mae: 0.0470
Epoch 247/500
2412/2412 [==============================] - 0s 98us/sample - loss: 0.0055 - mae: 0.0438
Epoch 248/500
2412/2412 [==============================] - 0s 97us/sample - loss: 

2412/2412 [==============================] - 0s 95us/sample - loss: 0.0044 - mae: 0.0393
Epoch 398/500
2412/2412 [==============================] - 0s 98us/sample - loss: 0.0043 - mae: 0.0389
Epoch 399/500
2412/2412 [==============================] - 0s 95us/sample - loss: 0.0046 - mae: 0.0414
Epoch 400/500
2412/2412 [==============================] - 0s 100us/sample - loss: 0.0044 - mae: 0.0393
Epoch 401/500
2412/2412 [==============================] - 0s 103us/sample - loss: 0.0046 - mae: 0.0402
Epoch 402/500
2412/2412 [==============================] - 0s 99us/sample - loss: 0.0043 - mae: 0.0380
Epoch 403/500
2412/2412 [==============================] - 0s 98us/sample - loss: 0.0040 - mae: 0.0366
Epoch 404/500
2412/2412 [==============================] - 0s 98us/sample - loss: 0.0038 - mae: 0.0344
Epoch 405/500
2412/2412 [==============================] - 0s 95us/sample - loss: 0.0044 - mae: 0.0398
Epoch 406/500
2412/2412 [==============================] - 0s 104us/sample - loss: 0.

...

2413/2413 [==============================] - 0s 122us/sample - loss: 0.0145 - mae: 0.0781
Epoch 56/500
2413/2413 [==============================] - 0s 131us/sample - loss: 0.0147 - mae: 0.0787
Epoch 57/500
2413/2413 [==============================] - 0s 122us/sample - loss: 0.0146 - mae: 0.0777
Epoch 58/500
2413/2413 [==============================] - 0s 115us/sample - loss: 0.0141 - mae: 0.0772
Epoch 59/500
2413/2413 [==============================] - 0s 119us/sample - loss: 0.0141 - mae: 0.0768
Epoch 60/500
2413/2413 [==============================] - 0s 123us/sample - loss: 0.0143 - mae: 0.0781
Epoch 61/500
2413/2413 [==============================] - 0s 128us/sample - loss: 0.0138 - mae: 0.0756
Epoch 62/500
2413/2413 [==============================] - 0s 140us/sample - loss: 0.0137 - mae: 0.0759
Epoch 63/500
2413/2413 [==============================] - 0s 125us/sample - loss: 0.0137 - mae: 0.0754
Epoch 64/500
2413/2413 [==============================] - 0s 128us/sample - loss: 0.01

2413/2413 [==============================] - 0s 93us/sample - loss: 0.0062 - mae: 0.0471
Epoch 214/500
2413/2413 [==============================] - 0s 93us/sample - loss: 0.0063 - mae: 0.0474
Epoch 215/500
2413/2413 [==============================] - 0s 105us/sample - loss: 0.0062 - mae: 0.0463
Epoch 216/500
2413/2413 [==============================] - 0s 103us/sample - loss: 0.0063 - mae: 0.0476
Epoch 217/500
2413/2413 [==============================] - 0s 97us/sample - loss: 0.0069 - mae: 0.0515
Epoch 218/500
2413/2413 [==============================] - 0s 92us/sample - loss: 0.0064 - mae: 0.0484
Epoch 219/500
2413/2413 [==============================] - 0s 102us/sample - loss: 0.0061 - mae: 0.0459
Epoch 220/500
2413/2413 [==============================] - 0s 107us/sample - loss: 0.0062 - mae: 0.0471
Epoch 221/500
2413/2413 [==============================] - 0s 115us/sample - loss: 0.0064 - mae: 0.0482
Epoch 222/500
2413/2413 [==============================] - 0s 115us/sample - loss:

2413/2413 [==============================] - 0s 93us/sample - loss: 0.0044 - mae: 0.0388
Epoch 372/500
2413/2413 [==============================] - 0s 102us/sample - loss: 0.0043 - mae: 0.0378
Epoch 373/500
2413/2413 [==============================] - 0s 107us/sample - loss: 0.0042 - mae: 0.0373
Epoch 374/500
2413/2413 [==============================] - 0s 110us/sample - loss: 0.0043 - mae: 0.0370
Epoch 375/500
2413/2413 [==============================] - 0s 102us/sample - loss: 0.0044 - mae: 0.0390
Epoch 376/500
2413/2413 [==============================] - 0s 96us/sample - loss: 0.0045 - mae: 0.0390
Epoch 377/500
2413/2413 [==============================] - 0s 104us/sample - loss: 0.0047 - mae: 0.0408
Epoch 378/500
2413/2413 [==============================] - 0s 103us/sample - loss: 0.0043 - mae: 0.0382
Epoch 379/500
2413/2413 [==============================] - 0s 106us/sample - loss: 0.0042 - mae: 0.0370
Epoch 380/500
2413/2413 [==============================] - 0s 108us/sample - los

2413/2413 [==============================] - 0s 100us/sample - loss: 0.0236 - mae: 0.0891
Epoch 30/500
2413/2413 [==============================] - 0s 110us/sample - loss: 0.0225 - mae: 0.0877
Epoch 31/500
2413/2413 [==============================] - 0s 106us/sample - loss: 0.0217 - mae: 0.0880
Epoch 32/500
2413/2413 [==============================] - 0s 97us/sample - loss: 0.0210 - mae: 0.0872
Epoch 33/500
2413/2413 [==============================] - 0s 106us/sample - loss: 0.0206 - mae: 0.0876
Epoch 34/500
2413/2413 [==============================] - 0s 102us/sample - loss: 0.0202 - mae: 0.0874
Epoch 35/500
2413/2413 [==============================] - 0s 101us/sample - loss: 0.0199 - mae: 0.0880
Epoch 36/500
2413/2413 [==============================] - 0s 109us/sample - loss: 0.0189 - mae: 0.0864
Epoch 37/500
2413/2413 [==============================] - 0s 106us/sample - loss: 0.0190 - mae: 0.0881
Epoch 38/500
2413/2413 [==============================] - 0s 98us/sample - loss: 0.0180

KeyboardInterrupt: 

(      no_mutations  line_coverage  isAssertionRoulette  isEagerTest  \
 0              580           0.51                    1            1   
 1              266           0.62                    1            1   
 2              581           1.00                    0            0   
 3              119           0.70                    0            1   
 4               21           1.00                    1            1   
 ...            ...            ...                  ...          ...   
 2678            85           0.88                    1            1   
 2679           751           0.70                    1            1   
 2680            95           0.85                    1            0   
 2681            14           1.00                    1            0   
 2682             2           0.50                    1            0   
 
       isLazyTest  isMysteryGuest  isSensitiveEquality  isResourceOptimism  \
 0              0               0                    0  

In [221]:
# Function to create model, required for KerasClassifier
def create_model(optimizer='adam', activation='linear', init_mode='uniform', dropout_rate=0.1):
    model = keras.Sequential()
    model.add(keras.layers.Dropout(dropout_rate, input_shape=(53,)))
    model.add(keras.layers.Dense(40, kernel_initializer=init_mode, activation=activation))
    model.add(keras.layers.Dense(20, kernel_initializer=init_mode, activation=activation))
    model.add(keras.layers.Dense(2, kernel_initializer=init_mode, activation='softmax'))

    model.compile(optimizer=optimizer,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
# load dataset
frame = load_frame()
frame = load_quartile(frame)

print(frame)

data_x, data_y, number_of_features = load_all_test_data_dynamic(frame) #load_all_prod_data(frame) #load_all_test_data_dynamic(frame) #load_all_prod_data(frame) #load_all_test_data_dynamic(frame)

data_x = data_x.values
data_y = data_y.values

scaler = StandardScaler()
scaler.fit(data_x)
data_x = scaler.transform(data_x)


early_stopping_monitor = keras.callbacks.EarlyStopping(monitor='accuracy', min_delta=0.0003, patience=10, verbose=0, mode='max', restore_best_weights=True)

model = KerasClassifier(build_fn=create_model, verbose=0, epochs=2000)

# define the grid search parameters
batch_size = [100]
activation = ['relu']
optimizer = ['Adam']
dropout_rate = [0.25]
param_grid = dict(batch_size=batch_size, optimizer=optimizer, activation=activation, dropout_rate=dropout_rate)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, cv=kfold, verbose=2)
grid_result = grid.fit(data_x, data_y, callbacks=[early_stopping_monitor])
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

/Users/Dor/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/Users/Dor/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


      mutation  no_mutations  line_coverage  isAssertionRoulette  isEagerTest  \
0            1            63       0.911765                    1            0   
1            0           235       0.430769                    1            1   
2            0           198       0.876923                    0            1   
3            1            24       0.866667                    0            0   
4            1             3       1.000000                    1            0   
...        ...           ...            ...                  ...          ...   
1322         1             4       1.000000                    1            0   
1323         0           269       0.466667                    1            1   
1324         0           384       0.978022                    0            1   
1325         0            44       0.750000                    1            1   
1326         1            30       1.000000                    1            0   

      isLazyTest  isMystery

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.8s remaining:    0.0s


[CV]  activation=relu, batch_size=100, dropout_rate=0.25, optimizer=Adam, total=   2.3s
[CV] activation=relu, batch_size=100, dropout_rate=0.25, optimizer=Adam 
[CV]  activation=relu, batch_size=100, dropout_rate=0.25, optimizer=Adam, total=   3.8s
[CV] activation=relu, batch_size=100, dropout_rate=0.25, optimizer=Adam 
[CV]  activation=relu, batch_size=100, dropout_rate=0.25, optimizer=Adam, total=   4.9s
[CV] activation=relu, batch_size=100, dropout_rate=0.25, optimizer=Adam 
[CV]  activation=relu, batch_size=100, dropout_rate=0.25, optimizer=Adam, total=   3.9s
[CV] activation=relu, batch_size=100, dropout_rate=0.25, optimizer=Adam 
[CV]  activation=relu, batch_size=100, dropout_rate=0.25, optimizer=Adam, total=   4.0s
[CV] activation=relu, batch_size=100, dropout_rate=0.25, optimizer=Adam 
[CV]  activation=relu, batch_size=100, dropout_rate=0.25, optimizer=Adam, total=   3.4s
[CV] activation=relu, batch_size=100, dropout_rate=0.25, optimizer=Adam 
[CV]  activation=relu, batch_size=

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   41.4s finished


Best: 0.807866 using {'activation': 'relu', 'batch_size': 100, 'dropout_rate': 0.25, 'optimizer': 'Adam'}
0.807866 (0.034673) with: {'activation': 'relu', 'batch_size': 100, 'dropout_rate': 0.25, 'optimizer': 'Adam'}
